Installa pacchetti aggiuntivi

<h1>Runnable GoogleColab project for testing the face_recognition library<h1>
<p>The following cell install the face_recognition library, essential to use this project, in google colab enviroment<p>

In [7]:
!pip install face_recognition
!pip install Pillow

     |████████████████████████████████| 100.1 MB 22 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=7f60cb0573c2afb3c512fc33d1454bbf9e57d6447cd3f4d637e0e06b6578afd0
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


<p>If you intend to use google drive for this project, run the following cell and replace every file path with "/content/drive/My Drive/file_name.png"<p>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# time test for face recognition and facial expressions (wink and smile)
import face_recognition
import cv2 as cv
# if not in google colab delete or comment the following line
from PIL import Image
import time
from scipy.spatial import distance as dist

def main():

    imagel = face_recognition.load_image_file("photos/biden.jpeg")
    imageb = face_recognition.load_image_file("photos/megan.jpg")
    try:
        biden_face_encoding = face_recognition.face_encodings(imagel)[0]
        megan_face_encoding = face_recognition.face_encodings(imageb)[0]
    except :
        print("I didn't any faces, quitting...\n")
        time.sleep(0.02)
        print(".\n")
        time.sleep(0.02)
        print(".\n")
        time.sleep(0.02)
        print(".\n")

    known_face_encodings = [
        biden_face_encoding,
        megan_face_encoding
    ]
    occhiolino = False
    sorriso = False
    sblocco = [False]
    t_start = 0
    t_sblocco = 0
    t_occhiolino = 0
    t_sorriso = 0
    # get the time
    t_start = time.time()
    file = "wink2.png" 
    # change the above string with the preferred string below:
    # "Winki.jpg" "biden.jpeg" "biden2.jpg" "biden3.jpg" "biden4.jpg" "biden5.jpg" "bush.jpg" 
    # "megan.jpg" "megan2.jpg" "megan3.jpg" "megan4.jpg" "not_smile.jpg" "wink2.png" 
    # note that only the files with "biden" or "megan will be recognized"

    image = cv.imread(f"photos/{file}")
    res_image = cv.resize(image, (0, 0), fx=0.5, fy=0.5)
    # get encoding from the image of the first face
    new_encoding = face_recognition.face_encodings(res_image)[0]
    # compoare with the known faces
    sblocco = face_recognition.compare_faces(known_face_encodings, new_encoding)
    # if there is at least one face that is recognized this will return True
    if True in sblocco: t_sblocco = time.time()
    # get the coordinates of the first face
    face_landmarks_list = face_recognition.face_landmarks(image)[0]
    for face_landmark in face_landmarks_list:
        # get the eyes
        occhiosx = face_landmark["left_eye"]
        occhiodx = face_landmark["right_eye"]
        # calculate the aspect ratio of the eyes
        apertura_sx = get_ear(occhiosx)
        apertura_dx = get_ear(occhiodx)
        # compare with stimated threshold
        occhiolino = (not (apertura_sx < 0.23) and (apertura_dx < 0.23)) or (not (apertura_dx < 0.23) and (apertura_sx < 0.23))
        # get the time fot the facial expr. wink
        if occhiolino: t_occhiolino = time.time()
        # get the two lips
        toplip = face_landmark["top_lip"]
        bottomlip = face_landmark["bottom_lip"]
        # check if the subject is smiling
        smiletl = get_smile_tl(toplip)
        smileop = get_smile_op(toplip, bottomlip)
        sorriso = smiletl or smileop
        # timer
        if sorriso: t_sorriso = time.time()
        t_end = time.time()
    
    # print of the results
    if True in sblocco: print(f"unlocked at:       {t_sblocco - t_start}") 
    else: print("error, subject not recognized")
    if occhiolino: print(f"wink at:                {t_occhiolino - t_start}") 
    else: print(f"error wink not found") 
    if sorriso: print(f"smile at:               {t_sorriso - t_start}") 
    else: print("error smile not found")
    print(f"test concluded in:      {t_end - t_start}")
    print(f"image dimensions:       {image.shape}")
    cv2_imshow(image)# if not in google colab comment this line and decomment the following
    # cv.imshow("image", image)
    cv.waitKey(0)
    cv.destroyAllWindows() 

def get_ear(eye):
    # this functon calculate the aspect ratio of the given eye
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    # return the eye aspect ratio
    return ear

    
def get_smile_tl(top_lip):
    # this function check if the borders of the mouth are higher than the center
    return top_lip[0][1]<top_lip[4][1] or top_lip[6][1]<top_lip[2][1] or top_lip[0][1]<top_lip[6][1] or top_lip[6][1]<top_lip[0][1]

def get_smile_op(top_lip, bottom_lip):
    # this function calculate the ascpect ratio of the mouth
    sar = dist.euclidean(top_lip[0],top_lip[6])/dist.euclidean(bottom_lip[9],top_lip[9])
    return sar < 5.5 and sar > 3

if __name__ == "__main__":
    main()
    

error, subject not recognized
wink at:                0.5039703845977783
smile at:               0.5039703845977783
test concluded in:      0.5039703845977783
image dimensions:       (512, 342, 3)
